In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

In [3]:
 # Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
 # create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
 # reflect an existing database into a new model
Base=automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# View all of the classes that automap found
 
Base.classes.keys()

['measurement', 'station']

In [8]:
# Save references to each table

Measurement=Base.classes.measurement

Station=Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
 
session=Session(engine)

In [20]:
# Find the most recent date in the data set. 

recent=session.query(Measurement).order_by(Measurement.date.desc())

for record in recent.limit(1).all():
    print(record.date)

2017-08-23


In [23]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
lastTwelve=session.query(Measurement).filter(Measurement.date<='2017-08-03').filter(Measurement.date>='2016-08-23')

# Perform a query to retrieve the data and precipitation scores
prcpList=[]
dateList=[]

for measurement in lastTwelve:
    dateList.append(measurement.date)
    prcpList.append(measurement.prcp)

In [40]:
# Save the query results as a Pandas DataFrame and set the index to the date column

frame={'prcp': prcpList, 'date': dateList}

dates=pd.DataFrame(frame).set_index('date')

# Sort the dataframe by date

dates.sort_values(by=['date'], axis=0)

,prcp
date,
2016-08-23,0.00
2016-08-23,NaN
2016-08-23,1.79
2016-08-23,0.05
2016-08-23,0.70
...,...
2017-08-02,0.25
2017-08-03,0.01
2017-08-03,0.00


In [39]:
# Use Pandas Plotting with Matplotlib to plot the data

,prcp
date,
2016-08-23,0.00
2016-08-23,NaN
2016-08-23,1.79
2016-08-23,0.05
2016-08-23,0.70
...,...
2017-08-02,0.25
2017-08-03,0.01
2017-08-03,0.00
